<a href="https://colab.research.google.com/github/june1st/GoogleColab/blob/master/MirrorGAN_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Tue May  7 07:17:09 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8    16W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
!git clone https://github.com/komiya-m/MirrorGAN
%cd MirrorGAN

Cloning into 'MirrorGAN'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 71 (delta 32), reused 54 (delta 15), pack-reused 0
Unpacking objects: 100% (71/71), done.
/content/MirrorGAN


In [3]:
!mkdir data
%cd data
#https://drive.google.com/open?id=1O_LtUP9sch09QH3s_EBAgLEctBQ5JBSJ
%env FILE_ID=1O_LtUP9sch09QH3s_EBAgLEctBQ5JBSJ
%env FILE_NAME=birds.zip
!curl -sc /tmp/cookie "https://drive.google.com/uc?export=download&id=${FILE_ID}" > /dev/null
!awk '/_warning_/ {print $NF}' /tmp/cookie > ./code.txt

test_data = open("code.txt", "r")
code = test_data.read().replace('\n','')
test_data.close()

import os
os.environ["CODE"] =code
!echo $CODE
!curl -Lb /tmp/cookie "https://drive.google.com/uc?export=download&confirm=${CODE}&id=${FILE_ID}" -o ${FILE_NAME}

#Download Images
!unzip birds.zip > /dev/null
%cd birds 
!wget http://www.vision.caltech.edu/visipedia-data/CUB-200-2011/CUB_200_2011.tgz
!tar -zxvf CUB_200_2011.tgz > /dev/null
%cd /content/MirrorGAN

/content/MirrorGAN/data
env: FILE_ID=1O_LtUP9sch09QH3s_EBAgLEctBQ5JBSJ
env: FILE_NAME=birds.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0     21      0 --:--:--  0:00:17 --:--:--    83
100 6336k    0 6336k    0     0   349k      0 --:--:--  0:00:18 --:--:-- 71.1M
/content/MirrorGAN/data/birds
--2019-05-07 07:18:23--  http://www.vision.caltech.edu/visipedia-data/CUB-200-2011/CUB_200_2011.tgz
Resolving www.vision.caltech.edu (www.vision.caltech.edu)... 34.208.54.77
Connecting to www.vision.caltech.edu (www.vision.caltech.edu)|34.208.54.77|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1150585339 (1.1G) [application/x-tar]
Saving to: ‘CUB_200_2011.tgz’

CUB_200_2011.tgz    100%[===================>]   1.07G  8.21MB/s    in 2m 7s   

2019-05-07 07:20:31 (8.64 MB/s) - ‘CUB_200_2011.tgz’ saved [1150585339/1150585339]

/

In [0]:
#cat config.py

In [4]:
%%writefile config.py
from __future__ import division
from __future__ import print_function

import os.path as osp
import numpy as np
from easydict import EasyDict as edict


__C = edict()
cfg = __C

# Dataset name: flowers, birds
__C.DATASET_NAME = 'birds'
__C.CONFIG_NAME = ''
__C.DATA_DIR = 'data/birds'

__C.RNN_TYPE = 'gru'  # 'lstm'

__C.TREE = edict()
__C.TREE.BRANCH_NUM = 3#1
__C.TREE.BASE_SIZE = 64

# Training options
__C.TRAIN = edict()
__C.TRAIN.BATCH_SIZE = 20
__C.TRAIN.MAX_EPOCH = 1
__C.TRAIN.FLAG = True
#loadするモデルのpath（重みのみ）
__C.TRAIN.NET_D = ''
__C.TRAIN.INIT_NET_G = ''
__C.TRAIN.NEXT128_NET_G = ''
__C.TRAIN.NEXT256_NET_G = ''
__C.TRAIN.RNN_DEC = ''#'model/cnn_rnn_encoder02_2.h5'

__C.TRAIN.D_LR = 0.0001 #Dのラーニングレート
__C.TRAIN.D_BETA1 = 0.5 #ADAM, beta1
__C.TRAIN.G_LR = 0.0004 #Gのラーニングレート
__C.TRAIN.G_BETA1 = 0.5 #ADAM, beta1
__C.TRAIN.RNN_DEC_LOSS_W = 0.1  #デコーダーRNNのloss_weight

__C.TRAIN.DEC_LR = 0.001  #pretrainのデコーダーRNNのラーニングレート
__C.TRAIN.DEC_SAVE_PATH = 'model/cnn_rnn_encoder.h5'
__C.TRAIN.DEC_MAX_EPOCH = 10

# Modal options
__C.GAN = edict()
__C.GAN.DF_DIM = 64
__C.GAN.GF_DIM = 128
__C.GAN.Z_DIM = 100
__C.GAN.CONDITION_DIM = 100
__C.GAN.R_NUM = 2
__C.GAN.B_ATTENTION = True
__C.GAN.B_DCGAN = False


__C.TEXT = edict()
__C.TEXT.CAPTIONS_PER_IMAGE = 10
__C.TEXT.EMBEDDING_DIM = 256
####追加
__C.TEXT.EMBEDDING_DIM_DEC = 512
__C.TEXT.HIDDEN_DIM = 256
####追加
__C.TEXT.HIDDEN_DIM_DEC = 512
__C.TEXT.WORDS_NUM = 18


def _merge_a_into_b(a, b):
    """Merge config dictionary a into config dictionary b, clobbering the
    options in b whenever they are also specified in a.
    """
    if type(a) is not edict:
        return

    for k, v in a.items():  #変更python3
        # a must specify keys that are in b
        if not k in b: #変更python3
            raise KeyError('{} is not a valid config key'.format(k))

        # the types must match, too
        old_type = type(b[k])
        if old_type is not type(v):
            if isinstance(b[k], np.ndarray):
                v = np.array(v, dtype=b[k].dtype)
            else:
                raise ValueError(('Type mismatch ({} vs. {}) '
                                  'for config key: {}').format(type(b[k]),
                                                               type(v), k))

        # recursively merge dicts
        if type(v) is edict:
            try:
                _merge_a_into_b(a[k], b[k])
            except:
                print('Error under config key: {}'.format(k))
                raise
        else:
            b[k] = v


def cfg_from_file(filename):
    """Load a config file and merge it into the default options."""
    import yaml
    with open(filename, 'r') as f:
        yaml_cfg = edict(yaml.load(f))

    _merge_a_into_b(yaml_cfg, __C)

Overwriting config.py


In [0]:
%mkdir model

In [0]:
run pretrain_STREAM.py

Using TensorFlow backend.


Total filenames:  11788 001.Black_footed_Albatross/Black_Footed_Albatross_0046_18.jpg
Load filenames from: data/birds/train/filenames.pickle (8855)
Load filenames from: data/birds/test/filenames.pickle (2933)
Load from:  data/birds/captions.pickle
Total filenames:  11788 001.Black_footed_Albatross/Black_Footed_Albatross_0046_18.jpg
Load filenames from: data/birds/train/filenames.pickle (8855)
Load filenames from: data/birds/test/filenames.pickle (2933)
Load from:  data/birds/captions.pickle
Instructions for updating:
Colocations handled automatically by placer.
87916544/87910968 [==============================] - 1s 0us/step
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
cr_pic_input (InputLayer)       (None, None, None, 3 0

In [0]:
%%writefile config.py
from __future__ import division
from __future__ import print_function

import os.path as osp
import numpy as np
from easydict import EasyDict as edict


__C = edict()
cfg = __C

# Dataset name: flowers, birds
__C.DATASET_NAME = 'birds'
__C.CONFIG_NAME = ''
__C.DATA_DIR = 'data/birds'

__C.RNN_TYPE = 'gru'  # 'lstm'

__C.TREE = edict()
__C.TREE.BRANCH_NUM = 1#1
__C.TREE.BASE_SIZE = 64

# Training options
__C.TRAIN = edict()
__C.TRAIN.BATCH_SIZE = 20
__C.TRAIN.MAX_EPOCH = 1
__C.TRAIN.FLAG = True
#loadするモデルのpath（重みのみ）
__C.TRAIN.NET_D = ''
__C.TRAIN.INIT_NET_G = ''
__C.TRAIN.NEXT128_NET_G = ''
__C.TRAIN.NEXT256_NET_G = ''
__C.TRAIN.RNN_DEC = 'model/cnn_rnn_encoder02_2.h5'

__C.TRAIN.D_LR = 0.0001 #Dのラーニングレート
__C.TRAIN.D_BETA1 = 0.5 #ADAM, beta1
__C.TRAIN.G_LR = 0.0004 #Gのラーニングレート
__C.TRAIN.G_BETA1 = 0.5 #ADAM, beta1
__C.TRAIN.RNN_DEC_LOSS_W = 0.1  #デコーダーRNNのloss_weight

__C.TRAIN.DEC_LR = 0.001  #pretrainのデコーダーRNNのラーニングレート
__C.TRAIN.DEC_SAVE_PATH = 'model/cnn_rnn_encoder.h5'
__C.TRAIN.DEC_MAX_EPOCH = 10

# Modal options
__C.GAN = edict()
__C.GAN.DF_DIM = 64
__C.GAN.GF_DIM = 128
__C.GAN.Z_DIM = 100
__C.GAN.CONDITION_DIM = 100
__C.GAN.R_NUM = 2
__C.GAN.B_ATTENTION = True
__C.GAN.B_DCGAN = False


__C.TEXT = edict()
__C.TEXT.CAPTIONS_PER_IMAGE = 10
__C.TEXT.EMBEDDING_DIM = 256
####追加
__C.TEXT.EMBEDDING_DIM_DEC = 512
__C.TEXT.HIDDEN_DIM = 256
####追加
__C.TEXT.HIDDEN_DIM_DEC = 512
__C.TEXT.WORDS_NUM = 18


def _merge_a_into_b(a, b):
    """Merge config dictionary a into config dictionary b, clobbering the
    options in b whenever they are also specified in a.
    """
    if type(a) is not edict:
        return

    for k, v in a.items():  #変更python3
        # a must specify keys that are in b
        if not k in b: #変更python3
            raise KeyError('{} is not a valid config key'.format(k))

        # the types must match, too
        old_type = type(b[k])
        if old_type is not type(v):
            if isinstance(b[k], np.ndarray):
                v = np.array(v, dtype=b[k].dtype)
            else:
                raise ValueError(('Type mismatch ({} vs. {}) '
                                  'for config key: {}').format(type(b[k]),
                                                               type(v), k))

        # recursively merge dicts
        if type(v) is edict:
            try:
                _merge_a_into_b(a[k], b[k])
            except:
                print('Error under config key: {}'.format(k))
                raise
        else:
            b[k] = v


def cfg_from_file(filename):
    """Load a config file and merge it into the default options."""
    import yaml
    with open(filename, 'r') as f:
        yaml_cfg = edict(yaml.load(f))

    _merge_a_into_b(yaml_cfg, __C)

In [8]:
run train.py

Using TensorFlow backend.


Total filenames:  11788 001.Black_footed_Albatross/Black_Footed_Albatross_0046_18.jpg
Load filenames from: data/birds/train/filenames.pickle (8855)
Load filenames from: data/birds/test/filenames.pickle (2933)


Instructions for updating:
Colocations handled automatically by placer.


Load from:  data/birds/captions.pickle


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


loadmodel_completed


  0%|          | 0/442 [00:00<?, ?it/s]

batch_size: 20  step_epoch : 442 srong_step_epoch 147
----------------EPOCH: 0 START----------------


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


ResourceExhaustedError: ignored